In [1]:
import warnings

warnings.filterwarnings("ignore")

import sys
import os

sys.path.append(os.path.abspath("../"))

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

import re
from tqdm import tqdm
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
DATA_PATH = "../sample_data/Bribes_Regression.dta"
df = pd.read_stata(DATA_PATH)
print(df.shape)

(2849, 57)


In [3]:
df.head(2).T

,0,1
ship_id,1.0,2.0
month_arrival,12.0,3.0
year,1.0,4.0
hc_group,15.0,12.0
hc_4digits,9405.0,8215.0
reason_bribe,congestion,undervalue
bp,NaN,0.0
actual_tariff_year,NaN,0.0
rsa,1,0
perishable,0.0,0.0


In [4]:
print("before dummies :", df.shape)

df = pd.concat(
    [
        df,
        pd.get_dummies(df["clear_agent"], drop_first=True, prefix="clear_agent"),
        pd.get_dummies(df["hc_group"], drop_first=True, prefix="hc_group"),
        pd.get_dummies(df["day_w_arrival"], drop_first=True, prefix="day_w_arrival"),
        pd.get_dummies(df["psi"], drop_first=True, prefix="psi"),
        pd.get_dummies(df["monitor"], drop_first=True, prefix="monitor")
    ],
    axis=1,
)
print("after dummies :", df.shape)

df.columns = [col.replace(".0", "") for col in df.columns]

before dummies : (2849, 57)
after dummies : (2849, 86)


In [5]:
print("before drop :", df.shape)
_df = df.dropna(subset=["clear_agent", "hc_group", "day_w_arrival", "psi", "monitor"]).drop(
    columns=["clear_agent", "hc_group", "day_w_arrival", "psi", "monitor"]
)
print("after drop :", _df.shape)

before drop : (2849, 86)
after drop : (2329, 81)


In [6]:
y_col = "lba"
d_col = "tariff_change_2008"
t_col = "post_2008"
X_cols = [
    "tariff2007",
    "lvalue_tonnage",
    "differentiated",
    "agri",
    "perishable",
    "dfs",
    "rsa",
    "hc_4digits"
]

x_pattern = re.compile(r'\Aclear_agent|\Ahc_group|\Aday_w_arrival|\Apsi|\Amonitor')
[X_cols.append(col) for col in _df.columns if bool(x_pattern.search(col) )]

X_cols 

['tariff2007',
 'lvalue_tonnage',
 'differentiated',
 'agri',
 'perishable',
 'dfs',
 'rsa',
 'hc_4digits',
 'day_w_arrival_post2008',
 'psi_post_2008',
 'monitor_tariff',
 'clear_agent_2',
 'clear_agent_3',
 'clear_agent_4',
 'clear_agent_5',
 'clear_agent_6',
 'clear_agent_7',
 'clear_agent_8',
 'hc_group_2',
 'hc_group_3',
 'hc_group_4',
 'hc_group_5',
 'hc_group_6',
 'hc_group_7',
 'hc_group_8',
 'hc_group_9',
 'hc_group_10',
 'hc_group_11',
 'hc_group_12',
 'hc_group_13',
 'hc_group_14',
 'hc_group_15',
 'day_w_arrival_Monday',
 'day_w_arrival_Saturday',
 'day_w_arrival_Sunday',
 'day_w_arrival_Thursday',
 'day_w_arrival_Tuesday',
 'day_w_arrival_Wednesday',
 'psi_not inspected',
 'monitor_Yes']

In [7]:
print("before NA drop", df.shape)
working_df = _df[[y_col, d_col, t_col] + X_cols].dropna()
print("after NA drop", working_df.shape)

before NA drop (2849, 86)
after NA drop (1084, 43)


In [8]:
model_df = working_df.copy()

In [113]:
from sklearn.model_selection import train_test_split
#from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.linear_model import LogisticRegressionCV, LassoCV, Lasso

In [10]:
K=2  # ２分割
df_set = train_test_split(model_df, random_state=1, test_size=0.5)
df_set[0][X_cols].head(3).T

,1126,2321,2266
tariff2007,0.000000,20.000000,5.000000
lvalue_tonnage,6.856199,10.108861,10.890318
differentiated,0.000000,1.000000,1.000000
agri,0.000000,1.000000,0.000000
perishable,0.000000,0.000000,0.000000
dfs,1.000000,0.000000,0.000000
rsa,1.000000,0.000000,0.000000
hc_4digits,4001.000000,8703.000000,8704.000000
day_w_arrival_post2008,3.000000,7.000000,2.000000
psi_post_2008,1.000000,1.000000,1.000000


In [11]:
d_model = LogisticRegressionCV(cv=5, random_state=0, penalty='l1', solver='saga')
d_model.fit(df_set[0][X_cols],  df_set[0][d_col])

LogisticRegressionCV(cv=5, penalty='l1', random_state=0, solver='saga')

In [12]:
# Output of d_model (cross-fitting).To prevent "division by zero", clip
eps = 0.03
ghat = np.clip(
    d_model.predict_proba(df_set[1][X_cols])[:, 1],
    eps,
    1 - eps,
)

In [13]:
lamda_hat = df_set[0][t_col].mean()
p_hat = df_set[0][d_col].mean()

control_y = df_set[0].query(f"{d_col} < 1")[y_col]
control_t = df_set[0].query(f"{d_col} < 1")[t_col]
control_x = df_set[0].query(f"{d_col} < 1")[X_cols]

_y = (control_t - lamda_hat )*control_y

In [14]:
l2k_model = LassoCV(cv=5, random_state=0)
l2k_model.fit(control_x, _y)

LassoCV(cv=5, random_state=0)

In [15]:
l2k_hat = l2k_model.predict(df_set[1][X_cols])

In [16]:
_c = (
    ((df_set[1][t_col] - lamda_hat) * df_set[1][y_col] - l2k_hat)
    * (df_set[1][d_col] - ghat)
    / ((1 - ghat) * lamda_hat * (1 - lamda_hat) * p_hat)
)

In [17]:
_c.mean()  # 論文の数式通りの実装

-17.277657442926387

In [18]:
_c[_c <= abs(_c.min())].mean()  # 著者の実装例をもとに後出しジャンケン

-17.277657442926387

In [52]:
eps = 0.03
cal_index= (eps<=ghat) & (ghat<=1-eps)
_c[cal_index]

2621      0.128679
203      -0.460959
2223     -0.166737
1299     -9.760081
835    -383.805075
           ...    
2684      0.278448
2566     -0.166737
483       5.405935
2665     -0.314446
2627      0.128679
Length: 542, dtype: float64

In [138]:
def DMLDiD_RSC(
    df,
    y_col,
    d_col,
    t_col,
    X_cols,
    ps_model=LogisticRegressionCV(cv=5, random_state=333, penalty="l1", solver="saga"),
    l2k_model=LassoCV(cv=5, random_state=333),
    original=True,
):
    B = 100
    K = 2  # ２分割

    thetabar = []
    thetabar_with_cap = []
    thetabar_3sigma = []

    for l in tqdm(range(B)):
        df_set = train_test_split(df, random_state=l + 1, test_size=0.5)
        _thetabar = []
        _thetabar_with_cap = []
        _thetabar_3sigma = []
        for i in range(K):
            k = 0 if i == 0 else 1
            c = 1 if i == 0 else 0

            d_model = ps_model
            d_model.fit(df_set[c][X_cols], df_set[c][d_col])

            eps = 0.03
            ghat = d_model.predict_proba(df_set[k][X_cols])[:, 1]

            cal_index = (eps <= ghat) & (ghat <= 1 - eps)
            #             ghat = np.clip(
            #                 d_model.predict_proba(df_set[k][X_cols])[:, 1],
            #                 eps,
            #                 1 - eps,
            #             )

            lamda_hat = df_set[c][t_col].mean()
            if original:
                p_hat = df_set[k][d_col].mean()  # 本当はｃでは？
            else:
                p_hat = df_set[c][d_col].mean()

            control_y = df_set[c].query(f"{d_col} < 1")[y_col]
            control_t = df_set[c].query(f"{d_col} < 1")[t_col]
            control_x = df_set[c].query(f"{d_col} < 1")[X_cols]

            _y = (control_t - lamda_hat) * control_y

            l2k_model = LassoCV(cv=5, random_state=333)
            l2k_model.fit(control_x, _y)

            if original:
                l2k_hat = df_set[k][X_cols] @ l2k_model.coef_  # 間違い　切片が消えてる
            else:
                l2k_hat = l2k_model.predict(df_set[k][X_cols])

            _c = (
                ((df_set[k][t_col] - lamda_hat) * df_set[k][y_col] - l2k_hat)
                * (df_set[k][d_col] - ghat)
                / ((1 - ghat) * lamda_hat * (1 - lamda_hat) * p_hat)
            )
            # ((T[index1]-lambda)*Y[index1]-ellhat2)*(D[index1]-ghat)/(1-ghat)/(lambda*(1-lambda))/mean(D[index1])
            _c = (
                ((df_set[k][t_col] - lamda_hat) * df_set[k][y_col] - l2k_hat)
                * (df_set[k][d_col] - ghat)
                / (1 - ghat)
                / (lamda_hat * (1 - lamda_hat))
                / p_hat
            )
            _c = _c[cal_index]

            _thetabar_with_cap.append(_c[_c <= abs(_c.min())].mean())
            _thetabar.append(_c.mean())

            up = _c.mean() + _c.std() * 3
            dwn = _c.mean() - _c.std() * 3
            _thetabar_3sigma.append(_c[(dwn <= _c) & (_c <= up)].mean())

        thetabar_with_cap.append(np.mean(_thetabar_with_cap))
        thetabar.append(np.mean(_thetabar))
        thetabar_3sigma.append(np.mean(_thetabar_3sigma))
    return np.mean(thetabar_with_cap), np.mean(thetabar), np.mean(thetabar_3sigma)

In [131]:
DMLDiD_RSC(model_df, y_col, d_col, t_col, X_cols)

100%|██████████| 100/100 [03:38<00:00,  2.18s/it]


(-15.442207185599475, -15.433295438330132)

In [30]:
%load_ext rpy2.ipython

%R library(readstata13)
#Raw Data
%R data <- read.dta13("../sample_data/Bribes_Regression.dta")

%R Y=data$lba
%R D=data$tariff_change_2008
%R T=data$post_2008

%R clear_agent2=(data$clear_agent==2)
%R clear_agent3=(data$clear_agent==3)
%R clear_agent4=(data$clear_agent==4)
%R clear_agent5=(data$clear_agent==5)
%R clear_agent6=(data$clear_agent==6)
%R clear_agent7=(data$clear_agent==7)
%R clear_agent8=(data$clear_agent==8)

%R hc_group2=(data$hc_group==2)
%R hc_group3=(data$hc_group==3)
%R hc_group4=(data$hc_group==4)
%R hc_group5=(data$hc_group==5)
%R hc_group6=(data$hc_group==6)
%R hc_group7=(data$hc_group==7)
%R hc_group8=(data$hc_group==8)
%R hc_group9=(data$hc_group==9)
%R hc_group10=(data$hc_group==10)
%R hc_group11=(data$hc_group==11)
%R hc_group12=(data$hc_group==12)
%R hc_group13=(data$hc_group==13)
%R hc_group14=(data$hc_group==14)
%R hc_group15=(data$hc_group==15)


%R X=cbind(data$post_2008, data$tariff2007,data$lvalue_tonnage, data$differentiated, data$agri, data$perishable,data$dfs, data$day_w_arrival, data$monitor, data$psi, data$rsa, data$post_2008,clear_agent2,clear_agent3,clear_agent4,clear_agent5,clear_agent6,clear_agent7,clear_agent8,hc_group2,hc_group3,hc_group4,hc_group5,hc_group6,hc_group7,hc_group8,hc_group9,hc_group10,hc_group11,hc_group12,hc_group13,hc_group14,hc_group15,data$hc_4digit)

#Working data
%R Working_data=cbind(Y,D,T,X)

%R Working_data=Working_data[complete.cases(Working_data), ]
%R -o Working_data

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [33]:
Working_data = pd.DataFrame(Working_data)


In [36]:
Working_data.columns = [f"v{i}" for i in range(37)]

In [37]:
Working_data

,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36
0,0.000000,1.0,1.0,1.0,20.0,6.239789,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8215.0
1,9.814213,0.0,1.0,1.0,20.0,10.423183,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8703.0
2,0.000000,0.0,1.0,1.0,20.0,9.832332,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8703.0
3,0.000000,0.0,1.0,1.0,7.5,3.246811,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7216.0
4,0.000000,0.0,1.0,1.0,20.0,4.179014,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,206.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,0.000000,0.0,1.0,1.0,20.0,9.650035,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8703.0
1080,0.000000,0.0,1.0,1.0,20.0,9.937679,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8703.0
1081,0.000000,0.0,1.0,1.0,20.0,10.343128,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8703.0
1082,0.000000,0.0,1.0,1.0,20.0,9.650020,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8703.0


In [128]:
_X = [col for col in Working_data.columns if col not in ["v0", "v1", "v2"]]
_X = ["v4", "v5", "v6", "v7", "v8", "v9", "v10", "v11"] # v3の情報が被っている、、、ここも間違っている

In [132]:
DMLDiD_RSC(Working_data, "v0", "v1","v2", _X)

100%|██████████| 100/100 [01:10<00:00,  1.42it/s]


(-25.861932834769814, -17.433583747391996)

In [134]:
DMLDiD_RSC(Working_data, "v0", "v1","v2", _X, original=False)

100%|██████████| 100/100 [01:10<00:00,  1.42it/s]


(-26.150625337158363, -17.907171440603474)

In [143]:
from lightgbm import LGBMClassifier, LGBMRegressor

DMLDiD_RSC(
    Working_data,
    "v0",
    "v1",
    "v2",
    _X,
    ps_model= LGBMClassifier( random_state=0),
    l2k_model=LGBMRegressor( random_state=0),
    original=False,
)

100%|██████████| 100/100 [00:27<00:00,  3.66it/s]


(-60.727326234268084, -6.442045260276781, -47.181341778944486)